# planMe

## Configuration

In [781]:
BASD = "20110606"
ETS = "20220827"
DOR = [
    ("E-3", BASD),
    ("E-4", "20130801"),
    ("E-5", "20140601"),
    ("E-6", "20180101"),
    ("W-1", "20211101"),
    ("W-2", "20231101"),
    ("W-3", "20281101"),
    ("W-4", "20341101")
]

BIRTHDAY = "19920821"

zipcode = "21144"
dependants = False

BRS = True

retirementYears = 20

# Format: (Name, Amount, Start date, End Date, Taxable),
AdditionalIncomes = [
    ("AIP", 1000, "20191101", "", False),
    ("SDAP", 150, "20191101", DOR[4][1], False),
    ("BAS", 375, BASD, DOR[4][1], False),
    ("BAS2", 258, DOR[4][1], "", False)
]

deductions = [
#     ("Alimony", 1200, "20200401", "20210801"),
]

# ( amount, Years)
reenlistmentBonuses = [
    (82000, "20210801", 6),
]

## Data Injest

In [782]:
taxBrackets = [
    (9875, .1),
    (40125, .12),
    (85525, .22),
    (163300, .24),
    (207350, .32),
    (518400, .35),
    (999999999, .37)
]

ranks = [
    "E-1",
    "E-2",
    "E-3",
    "E-4",
    "E-5",
    "E-6",
    "E-7",
    "E-8",
    "E-9",
    "W-1",
    "W-2",
    "W-3",
    "W-4",
    "W-5",
    "O-1E",
    "O-2E",
    "O-3E",
    "O-1",
    "O-2",
    "O-3",
    "O-4",
    "O-5",
    "O-6",
    "O-7"
]

In [783]:
print(len(ranks))

24


In [784]:
with open("withDepend.txt", "r") as f:
    withDependstr = f.read()

withDepend = withDependstr.strip().split('\n')

In [785]:
withDependComb = {}

for line in withDepend:
    if line.startswith("ZZ"):
        continue
    left = line.split()
    mha = left[0]
    withDependComb[mha] = {}
    i = 0
    for cell in left[1:]:
        if cell.isnumeric():
            try:
                withDependComb[mha][ranks[i]] = float(cell)
            except Exception as e:
                print(e)
                print(cell)
                print(i)
                print(line)
            i += 1

In [786]:
with open("withoutDepend.txt.new", "r") as f:             
    withoutDependstr = f.read()

withoutDepend = withoutDependstr.strip().split('\n')

In [787]:
withoutDependComb = {}

for line in withoutDepend:
    if line.startswith("ZZ"):
        continue
    left = line.split()
    mha = left[0]
    withoutDependComb[mha] = {}
    i = 0
    for cell in left[1:]:
        if cell.isnumeric():
            withoutDependComb[mha][ranks[i]] = float(cell)
            i += 1

In [788]:
mhaCodes = {}
with open("BAH2021/sorted_zipmha21.txt") as f:
    for line in f:
        zipCode, mha = line.split()
        mhaCodes[zipCode] = mha

### Format Military Pay Scale

In [789]:
with open("payscale.txt", "r") as f:
    milPayScaleStr = f.read()

In [790]:
milPay = dict()
for line in milPayScaleStr.strip().split("\n")[1:]:
    row = line.split()
    row[0] = row[0].replace('*',"")
    milPay[row[0]] = []
    j = 0
    prev = 0.0
    for val in row[1:]:
        if val == ">4" or val == "<4" or val == "Mon":
            val = row[3]
        fVal = float(val)
        if j == 0:
            milPay[row[0]].append(fVal)
            milPay[row[0]].append(fVal)
            j=1
        elif j < 4:
            milPay[row[0]].append(fVal)
            j+=1
        else:
            milPay[row[0]].append(fVal)
            milPay[row[0]].append(fVal)
            j+=2

### Generate Monthly Income

Fix Date and times

In [791]:
from datetime import datetime as dt

fmt = "%Y%m%d"

basd = dt.strptime(BASD, fmt).replace(day=1)
ets = dt.strptime(ETS, fmt).replace(day=1)
birthday = dt.strptime(BIRTHDAY, fmt).replace(day=1)
dor = {}
for rank, date in DOR:
    dor[dt.strptime(date, fmt).replace(day=1)] = rank


addtlPays = []
for _, amount, start, end, tax in AdditionalIncomes:
    if end == "":
        addtlPays.append((amount, dt.strptime(start, fmt), retirement, tax))
    else:
        addtlPays.append((amount, dt.strptime(start, fmt), dt.strptime(end, fmt), tax))
        
deduct = []
for _, amount, start, end in deductions:
    if end == "":
        deduct.append((amount, dt.strptime(start, fmt), retirement))
    else:
        deduct.append((amount, dt.strptime(start, fmt), dt.strptime(end, fmt)))
        
bonuss = []
for amount, start, yrs in reenlistmentBonuses:
    bonuss.append((amount, dt.strptime(start, fmt), yrs))
    

Create Retirement date

In [792]:
from datetime import timedelta
retirement = basd + timedelta(days=(retirementYears*365))
retirement = retirement.replace(day=1)

Create Death at 90 years old

In [793]:
death = birthday + timedelta(days = (90 *365))
death = death.replace(day=1)

Generate Months

In [794]:
from dateutil import rrule
from dateutil.relativedelta import relativedelta

months = {}

curRank = dor[basd]
for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=death):
        months[month] = 0

Add Base Pay

In [795]:
curRank = dor[basd]
for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=retirement):
    yos = relativedelta(month, basd).years
    if month in dor:
        curRank = dor[month]
    months[month] = milPay[curRank][yos]

for item in months.items():
    print(item)

Generate Pension from Base Pay

In [796]:
high3Start = retirement + relativedelta(years=-3)
high3Start = high3Start.replace(day=1)

In [797]:
total = 0
for month in rrule.rrule(rrule.MONTHLY, dtstart=high3Start, until=retirement):
    total += months[month]

Calculate pension multiplyer

In [798]:
mul = .025 if BRS else .02

In [799]:
pension = (total / 36) * (mul * retirementYears)

Add Pension to months after retirement

In [800]:
pensionStart = retirement + relativedelta(months=+1)
pensionStart = pensionStart.replace(day=1)
for month in rrule.rrule(rrule.MONTHLY, dtstart=pensionStart, until=death):
    months[month] += pension

Add additonal duty pays

In [801]:
for amount, start, end, tax in addtlPays:
    for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=retirement):
        if month >= start and (end == "" or month < end):
            months[month] += amount

Add Bonus'

In [802]:
for amount, date, yrs in bonuss:
    monthly = (amount)/(yrs * 12)
    end = date + relativedelta(months=+(yrs * 12))
    for month in rrule.rrule(rrule.MONTHLY, dtstart=date, until=end):
        months[month] += monthly

Remove Taxes

In [803]:
for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=retirement):
    amount = months[month]
    anual = amount * 12
    owed = 0
    for cap, per in taxBrackets:
        if anual < 0:
            break
        if anual > cap:
            owed = cap * per
            anual -= cap
        elif anual < cap:
            owed = anual * per
            break
    months[month] -= owed / 12

Add BAH Rates

In [804]:
curRank = dor[basd]
for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=retirement):
    yos = relativedelta(month, basd).years
    if month in dor:
        curRank = dor[month]
    mhaCode = mhaCodes[zipcode]
    if dependants:
        months[month] += withDependComb[mhaCode][curRank]
    else:
        months[month] += withoutDependComb[mhaCode][curRank]

Remove deductions

In [805]:
for amount, start, end in deduct:
    for month in rrule.rrule(rrule.MONTHLY, dtstart=basd, until=retirement):
        if month >= start and (end == "" or month < end):
            months[month] -= amount

## Display Data

### Stay until Retirement

In [806]:
# for item in months.items():
#     print(item)

### Get Out at current ETS

In [807]:
yearlyInterstGoal = pension * 12

neededInvestment = yearlyInterstGoal / .08

growthTime = retirement - ets

In [808]:
yearsToGrow = int(abs(growthTime.days / 365)) + 1

In [809]:
monthlyInvestment = 0
i = 0
while(True):
    total = 0
    for j in range(0, yearsToGrow):
        total = (total + (i * 12)) * 1.08
    if total > neededInvestment:
        monthlyInvestment = i
        break
    i += 1

In [810]:
print(monthlyInvestment)

3147


Add the monthly Investment required to income after ETS before Retirement

In [811]:
outMonths = {}
print(f"ets {ets} ret {retirement}")
for month in rrule.rrule(rrule.MONTHLY, dtstart=ets, until=retirement):
    outMonths[month] = months[month] + monthlyInvestment

ets 2022-08-01 00:00:00 ret 2031-05-01 00:00:00


Aproximate the Anual income required to match the income of military service while investing to generate enough money to create an equalivent "pension"

In [812]:
for year in rrule.rrule(rrule.YEARLY,  dtstart=ets, until=retirement):
    anual = outMonths[year] * 12
    owed = 0
    for cap, per in taxBrackets:
        if anual < 0:
            break
        if anual > cap:
            owed += cap * per
            anual -= cap
        elif anual < cap:
            owed += anual * per
            break
    anual = outMonths[year] * 12
    print(f"Anual Income required: ${anual + owed:.2f}")

Anual Income required: $168731.62
Anual Income required: $168731.62
Anual Income required: $178169.05
Anual Income required: $178169.05
Anual Income required: $180226.86
Anual Income required: $180226.86
Anual Income required: $168867.81
Anual Income required: $179372.40
Anual Income required: $182460.87


In [813]:
# for item in months.items():
#     print(item)